TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 21
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 31 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.915107,2.305036,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,31.796849,0,3.913019
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.922088,0,3.271547
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.805147,0,2.280972
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,23.862167,3,2.686681
5,6.186810,1.546702,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,17.656299,1,1.924635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,42.800000,3.057143,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.4,2.4,7.8,2.0,6.400,25.495576,1,2.898988
611,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.329379,0,3.353084
621,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.329379,0,3.353084


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 31 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.915107,2.305036,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,31.796849,0,3.913019
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,23.862167,3,2.686681
5,6.186810,1.546702,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.000000,NaN,NaN,NaN,NaN,NaN,17.656299,1,1.924635
6,45.753000,3.050200,3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,29.343168,0,3.269825
7,19.800256,1.980026,1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,19.555100,4,2.329895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,9.300000,1.033333,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,16.627191,0,1.863502
461,45.766518,3.051101,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,26.620096,0,2.868422
470,32.130643,3.570071,1,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,27.017771,1,2.967257


In [5]:
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 76.0
Expected points: 104.92177438735962


,element_type,web_name,now_cost,games played,gameweek 22 prediction
id,,,,,
389,1,Bachmann,43,10.0,8.072920
135,2,Chilwell,56,6.0,8.417256
367,2,Sánchez,46,12.0,8.322935
370,2,Reguilón,54,17.0,8.508138
138,3,Mount,77,18.0,8.248278
210,3,Tielemans,64,14.0,7.901112
212,3,Maddison,69,18.0,8.732699
359,3,Son,107,17.0,10.212366
450,4,Dennis,61,17.0,8.842440


In [6]:
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 85.2
Expected points per week: 74.1156914554352


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
376,1,Foster,41,8.0,3.800216
122,2,Alonso,57,15.0,5.008928
135,2,Chilwell,56,6.0,6.133228
237,2,Alexander-Arnold,84,18.0,6.818443
256,2,Cancelo,70,20.0,5.864771
233,3,Salah,128,20.0,7.670902
240,3,Jota,82,19.0,5.696192
265,3,Foden,81,13.0,5.894807
268,3,Torres,67,4.0,6.321480


In [7]:
df = df.drop([135, 268])
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 84.60000000000001
Expected points per week: 71.85129562210187


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
376,1,Foster,41,8.0,3.800216
122,2,Alonso,57,15.0,5.008928
142,2,James,62,16.0,5.037333
237,2,Alexander-Arnold,84,18.0,6.818443
256,2,Cancelo,70,20.0,5.864771
233,3,Salah,128,20.0,7.670902
240,3,Jota,82,19.0,5.696192
265,3,Foden,81,13.0,5.894807
266,3,Rodrigo,55,12.0,5.152980


In [8]:
optimize(df, 85.2, 'xPoints_next10')

Status: Optimal
Cost: 82.9
Expected points per week: 58.08539178371429


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
556,1,Fernández,45,10.0,44.989489
237,2,Alexander-Arnold,84,18.0,54.348533
256,2,Cancelo,70,20.0,44.597128
367,2,Sánchez,46,12.0,44.668084
370,2,Reguilón,54,17.0,46.186502
212,3,Maddison,69,18.0,45.391452
233,3,Salah,128,20.0,53.368315
265,3,Foden,81,13.0,47.344779
359,3,Son,107,17.0,56.086877
